In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/2D_Pixelated_designs_Static_Combined - Data.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
len(df['geometry_full'][0])

In [ ]:
df.shape

In [ ]:
sub = df.sample(frac = 0.002)
sub.to_csv('data/sample.csv')

In [ ]:
sub.shape

In [ ]:
def transform(itr,tmx):
    '''
    Transform 3D-tensor (Euclidean or Cartesion tensor) of any order (>0) to another coordinate system
    Parameters:
        itr: input tensor, before transformation; should be a 3-element vector, a 3x3 matrix, or a 3x3x3x... multidimensional array, each dimension containing 3 elements
        tmx: transformation matrix, 3x3 matrix that contains the direction cosines between the old and the new coordinate system
    Returns:
        otr: output tensor, after transformation; has the same dimensions as the input tensor
    '''
    ne = np.asarray(itr).size
    nd = np.asarray(itr).ndim
    
    if (ne == 3):
        nd = 1
    
    otr = np.zeros_like(itr)
    
    iie = np.zeros((nd,1))
    ioe = np.zeros((nd,1))
    cne = np.cumprod(3 * np.ones((nd,1))) / 3
    
    for oe in range(ne):
        ioe = np.mod(np.floor(oe / cne).astype(int), 3)
        for ie in range(ne):
            pmx = 1
            iie = np.mod(np.floor(ie / cne).astype(int), 3)
            for i in range(nd):
                pmx = pmx * tmx[ioe[i], iie[i]]
            otr[np.unravel_index(oe, otr.shape)] = otr[np.unravel_index(oe, otr.shape)] + pmx * itr[np.unravel_index(ie, itr.shape)]
    
    return otr

In [ ]:
def visual2D(c): 
    '''

    Parameters
    ----------
    c : 1D numpy array
        [C11, C12, C22, C16, C26, C66].

    Returns
    -------
    None.

    '''
    # transform c to CH (a 6x6 matrix)
    CH = np.ones((6,6)) * 1e-7
    CH[0,0] = c[0]
    CH[0,1] = c[1]
    CH[1,0] = c[1]
    CH[1,1] = c[2]
    CH[0,5] = c[3]
    CH[5,0] = c[3]
    CH[1,5] = c[4]
    CH[5,1] = c[4]
    CH[5,5] = c[5]
    # transform CH to a 3*3*3*3 tensor
    tensor = generate(CH)
    # find the E1 in 360 degree
    a = np.arange(0, 2*np.pi+0.02*np.pi, 0.02*np.pi)
    E1 = np.zeros_like(a)
    P1 = np.zeros_like(a)
    print(len(a))
    for i in range(len(a)):
        trans_z = np.array([[np.cos(a[i]), -np.sin(a[i]), 0],
                            [np.sin(a[i]),  np.cos(a[i]), 0],
                            [0,             0,            1]])
        # calculate the new tensor
        N_tensor = transform(tensor, trans_z)
        # transform the tensor to 6*6
        N_CH = ToMatrix(N_tensor)
        # calculate the E1
        E, P = modulus(N_CH)
        E1[i] = E[0]
        P1[i] = P[0,1]
    # x,y = pol2cart(a,E1)
#     print(E1, P1, E1.shape, P1.shape)
    return E1, P1
#     # Plot figure
#     fig, axs = plt.subplots(1, 2, subplot_kw={'projection': 'polar'})
#     axs[0].plot(a, E1)
#     axs[0].grid(True)
#     axs[0].set_title("Young's Modulus")
#     axs[1].plot(a, P1)
#     axs[1].grid(True)
#     axs[1].set_title("Poisson's Ratio")
#     plt.tight_layout()
#     plt.show()
    
    
def modulus(CH = None): 
    S = np.linalg.inv(CH[np.ix_([0,1,3], [0,1,3])])
    E = np.zeros((3,1))
    P = np.zeros((3,3))
    E[0] = 1 / S[0,0]
    E[1] = 1 / S[1,1]
    E[2] = 1 / S[2,2]
    P[0,1] = -S[0,1] * E[1];
    P[1,0] = -S[1,0] * E[0];
    P[2,1] = -S[1,2] * E[1];
    P[1,2] = -S[2,1] * E[2];
    return E, P
    
    
def generate(CH = None): 
    C = np.zeros((3,3,3,3))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            C[a,b,c,d] = CH[i,j]
    for i in range(3):
        if (i == 2):
            j = 0
        else:
            j = i + 1
        for m in range(3):
            if (m == 2):
                n = 0
            else:
                n = m + 1
            C[j,i,n,m] = C[i,j,m,n]
            C[j,i,m,n] = C[i,j,m,n]
            C[i,j,n,m] = C[i,j,m,n]
            C[j,i,m,m] = C[i,j,m,m]
            C[m,m,j,i] = C[m,m,i,j]
    return C
    
    
def change(w = None): 
    # change the index 4 5 6 to 23 31 12
    if (w < 3):
        a = w
        b = w
    else:
        if (w == 3):
            a = 1
            b = 2
        else:
            if (w == 4):
                a = 2
                b = 0
            else:
                if (w == 5):
                    a = 0
                    b = 1
    return a,b
    
    
def ToMatrix(C = None): 
    CH = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            CH[i,j] = C[a,b,c,d]
    return CH

In [ ]:
import time 

start = time.time() 
c = np.array([604591262.41, 102330968.98, 1157840534.65, 0.00, 0.00, 156833419.95])
visual2D(c)
print(time.time() - start)

In [ ]:
small = sub.head(n=3)

In [ ]:
#[C11, C12, C22, C16, C26, C66]
start = time.time()
res = small.apply(lambda row : visual2D([row['C11'], row["C12"], row['C22'], row['C16'], row['C26'], row['C66']]), axis=1)
print("Time for computing %d rows: %f"%(sub.shape[0], time.time() - start))


In [ ]:
sub['youngs'] = res.apply(lambda x : x[0])
sub['poisson'] = res.apply(lambda x : x[1])

In [ ]:
sub.head()

In [ ]:
sub.to_csv('data/smallsample.csv')

In [ ]:
sub = df.sample(frac = 0.002)
sub.to_csv('data/sample.csv')

In [ ]:
sub.shape

In [ ]:
def transform(itr,tmx):
    '''
    Transform 3D-tensor (Euclidean or Cartesion tensor) of any order (>0) to another coordinate system
    Parameters:
        itr: input tensor, before transformation; should be a 3-element vector, a 3x3 matrix, or a 3x3x3x... multidimensional array, each dimension containing 3 elements
        tmx: transformation matrix, 3x3 matrix that contains the direction cosines between the old and the new coordinate system
    Returns:
        otr: output tensor, after transformation; has the same dimensions as the input tensor
    '''
    ne = np.asarray(itr).size
    nd = np.asarray(itr).ndim
    
    if (ne == 3):
        nd = 1
    
    otr = np.zeros_like(itr)
    
    iie = np.zeros((nd,1))
    ioe = np.zeros((nd,1))
    cne = np.cumprod(3 * np.ones((nd,1))) / 3
    
    for oe in range(ne):
        ioe = np.mod(np.floor(oe / cne).astype(int), 3)
        for ie in range(ne):
            pmx = 1
            iie = np.mod(np.floor(ie / cne).astype(int), 3)
            for i in range(nd):
                pmx = pmx * tmx[ioe[i], iie[i]]
            otr[np.unravel_index(oe, otr.shape)] = otr[np.unravel_index(oe, otr.shape)] + pmx * itr[np.unravel_index(ie, itr.shape)]
    
    return otr

In [ ]:
def visual2D(c): 
    '''

    Parameters
    ----------
    c : 1D numpy array
        [C11, C12, C22, C16, C26, C66].

    Returns
    -------
    None.

    '''
    # transform c to CH (a 6x6 matrix)
    CH = np.ones((6,6)) * 1e-7
    CH[0,0] = c[0]
    CH[0,1] = c[1]
    CH[1,0] = c[1]
    CH[1,1] = c[2]
    CH[0,5] = c[3]
    CH[5,0] = c[3]
    CH[1,5] = c[4]
    CH[5,1] = c[4]
    CH[5,5] = c[5]
    # transform CH to a 3*3*3*3 tensor
    tensor = generate(CH)
    # find the E1 in 360 degree
    a = np.arange(0, 2*np.pi+0.02*np.pi, 0.02*np.pi)
    E1 = np.zeros_like(a)
    P1 = np.zeros_like(a)
    print(len(a))
    for i in range(len(a)):
        trans_z = np.array([[np.cos(a[i]), -np.sin(a[i]), 0],
                            [np.sin(a[i]),  np.cos(a[i]), 0],
                            [0,             0,            1]])
        # calculate the new tensor
        N_tensor = transform(tensor, trans_z)
        # transform the tensor to 6*6
        N_CH = ToMatrix(N_tensor)
        # calculate the E1
        E, P = modulus(N_CH)
        E1[i] = E[0]
        P1[i] = P[0,1]
    # x,y = pol2cart(a,E1)
#     print(E1, P1, E1.shape, P1.shape)
    return E1, P1
#     # Plot figure
#     fig, axs = plt.subplots(1, 2, subplot_kw={'projection': 'polar'})
#     axs[0].plot(a, E1)
#     axs[0].grid(True)
#     axs[0].set_title("Young's Modulus")
#     axs[1].plot(a, P1)
#     axs[1].grid(True)
#     axs[1].set_title("Poisson's Ratio")
#     plt.tight_layout()
#     plt.show()
    
    
def modulus(CH = None): 
    S = np.linalg.inv(CH[np.ix_([0,1,3], [0,1,3])])
    E = np.zeros((3,1))
    P = np.zeros((3,3))
    E[0] = 1 / S[0,0]
    E[1] = 1 / S[1,1]
    E[2] = 1 / S[2,2]
    P[0,1] = -S[0,1] * E[1];
    P[1,0] = -S[1,0] * E[0];
    P[2,1] = -S[1,2] * E[1];
    P[1,2] = -S[2,1] * E[2];
    return E, P
    
    
def generate(CH = None): 
    C = np.zeros((3,3,3,3))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            C[a,b,c,d] = CH[i,j]
    for i in range(3):
        if (i == 2):
            j = 0
        else:
            j = i + 1
        for m in range(3):
            if (m == 2):
                n = 0
            else:
                n = m + 1
            C[j,i,n,m] = C[i,j,m,n]
            C[j,i,m,n] = C[i,j,m,n]
            C[i,j,n,m] = C[i,j,m,n]
            C[j,i,m,m] = C[i,j,m,m]
            C[m,m,j,i] = C[m,m,i,j]
    return C
    
    
def change(w = None): 
    # change the index 4 5 6 to 23 31 12
    if (w < 3):
        a = w
        b = w
    else:
        if (w == 3):
            a = 1
            b = 2
        else:
            if (w == 4):
                a = 2
                b = 0
            else:
                if (w == 5):
                    a = 0
                    b = 1
    return a,b
    
    
def ToMatrix(C = None): 
    CH = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            CH[i,j] = C[a,b,c,d]
    return CH

In [ ]:
import time 

start = time.time() 
c = np.array([604591262.41, 102330968.98, 1157840534.65, 0.00, 0.00, 156833419.95])
visual2D(c)
print(time.time() - start)

In [ ]:
small = sub.head(n=3)

In [ ]:
#[C11, C12, C22, C16, C26, C66]
start = time.time()
res = small.apply(lambda row : visual2D([row['C11'], row["C12"], row['C22'], row['C16'], row['C26'], row['C66']]), axis=1)
print("Time for computing %d rows: %f"%(sub.shape[0], time.time() - start))


In [ ]:
sub['youngs'] = res.apply(lambda x : x[0])
sub['poisson'] = res.apply(lambda x : x[1])

In [ ]:
sub.head()

In [ ]:
sub.to_csv('data/smallsample2.csv')

In [ ]:
sub = df.sample(frac = 0.002)
sub.to_csv('data/sample.csv')

In [ ]:
sub.shape

In [ ]:
def transform(itr,tmx):
    '''
    Transform 3D-tensor (Euclidean or Cartesion tensor) of any order (>0) to another coordinate system
    Parameters:
        itr: input tensor, before transformation; should be a 3-element vector, a 3x3 matrix, or a 3x3x3x... multidimensional array, each dimension containing 3 elements
        tmx: transformation matrix, 3x3 matrix that contains the direction cosines between the old and the new coordinate system
    Returns:
        otr: output tensor, after transformation; has the same dimensions as the input tensor
    '''
    ne = np.asarray(itr).size
    nd = np.asarray(itr).ndim
    
    if (ne == 3):
        nd = 1
    
    otr = np.zeros_like(itr)
    
    iie = np.zeros((nd,1))
    ioe = np.zeros((nd,1))
    cne = np.cumprod(3 * np.ones((nd,1))) / 3
    
    for oe in range(ne):
        ioe = np.mod(np.floor(oe / cne).astype(int), 3)
        for ie in range(ne):
            pmx = 1
            iie = np.mod(np.floor(ie / cne).astype(int), 3)
            for i in range(nd):
                pmx = pmx * tmx[ioe[i], iie[i]]
            otr[np.unravel_index(oe, otr.shape)] = otr[np.unravel_index(oe, otr.shape)] + pmx * itr[np.unravel_index(ie, itr.shape)]
    
    return otr

In [ ]:
def visual2D(c): 
    '''

    Parameters
    ----------
    c : 1D numpy array
        [C11, C12, C22, C16, C26, C66].

    Returns
    -------
    None.

    '''
    # transform c to CH (a 6x6 matrix)
    CH = np.ones((6,6)) * 1e-7
    CH[0,0] = c[0]
    CH[0,1] = c[1]
    CH[1,0] = c[1]
    CH[1,1] = c[2]
    CH[0,5] = c[3]
    CH[5,0] = c[3]
    CH[1,5] = c[4]
    CH[5,1] = c[4]
    CH[5,5] = c[5]
    # transform CH to a 3*3*3*3 tensor
    tensor = generate(CH)
    # find the E1 in 360 degree
    a = np.arange(0, 2*np.pi+0.02*np.pi, 0.02*np.pi)
    E1 = np.zeros_like(a)
    P1 = np.zeros_like(a)
    print(len(a))
    for i in range(len(a)):
        trans_z = np.array([[np.cos(a[i]), -np.sin(a[i]), 0],
                            [np.sin(a[i]),  np.cos(a[i]), 0],
                            [0,             0,            1]])
        # calculate the new tensor
        N_tensor = transform(tensor, trans_z)
        # transform the tensor to 6*6
        N_CH = ToMatrix(N_tensor)
        # calculate the E1
        E, P = modulus(N_CH)
        E1[i] = E[0]
        P1[i] = P[0,1]
    # x,y = pol2cart(a,E1)
#     print(E1, P1, E1.shape, P1.shape)
    return E1, P1
#     # Plot figure
#     fig, axs = plt.subplots(1, 2, subplot_kw={'projection': 'polar'})
#     axs[0].plot(a, E1)
#     axs[0].grid(True)
#     axs[0].set_title("Young's Modulus")
#     axs[1].plot(a, P1)
#     axs[1].grid(True)
#     axs[1].set_title("Poisson's Ratio")
#     plt.tight_layout()
#     plt.show()
    
    
def modulus(CH = None): 
    S = np.linalg.inv(CH[np.ix_([0,1,3], [0,1,3])])
    E = np.zeros((3,1))
    P = np.zeros((3,3))
    E[0] = 1 / S[0,0]
    E[1] = 1 / S[1,1]
    E[2] = 1 / S[2,2]
    P[0,1] = -S[0,1] * E[1];
    P[1,0] = -S[1,0] * E[0];
    P[2,1] = -S[1,2] * E[1];
    P[1,2] = -S[2,1] * E[2];
    return E, P
    
    
def generate(CH = None): 
    C = np.zeros((3,3,3,3))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            C[a,b,c,d] = CH[i,j]
    for i in range(3):
        if (i == 2):
            j = 0
        else:
            j = i + 1
        for m in range(3):
            if (m == 2):
                n = 0
            else:
                n = m + 1
            C[j,i,n,m] = C[i,j,m,n]
            C[j,i,m,n] = C[i,j,m,n]
            C[i,j,n,m] = C[i,j,m,n]
            C[j,i,m,m] = C[i,j,m,m]
            C[m,m,j,i] = C[m,m,i,j]
    return C
    
    
def change(w = None): 
    # change the index 4 5 6 to 23 31 12
    if (w < 3):
        a = w
        b = w
    else:
        if (w == 3):
            a = 1
            b = 2
        else:
            if (w == 4):
                a = 2
                b = 0
            else:
                if (w == 5):
                    a = 0
                    b = 1
    return a,b
    
    
def ToMatrix(C = None): 
    CH = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            CH[i,j] = C[a,b,c,d]
    return CH

In [ ]:
import time 

start = time.time() 
c = np.array([604591262.41, 102330968.98, 1157840534.65, 0.00, 0.00, 156833419.95])
visual2D(c)
print(time.time() - start)

In [ ]:
small = sub.head(n=3)

In [ ]:
#[C11, C12, C22, C16, C26, C66]
start = time.time()
res = small.apply(lambda row : visual2D([row['C11'], row["C12"], row['C22'], row['C16'], row['C26'], row['C66']]), axis=1)
print("Time for computing %d rows: %f"%(sub.shape[0], time.time() - start))


In [ ]:
sub['youngs'] = res.apply(lambda x : x[0])
sub['poisson'] = res.apply(lambda x : x[1])

In [ ]:
sub.head()

In [ ]:
sub.to_csv('data/smallsample2.csv')

In [64]:
sub.shape

(158, 26)

In [65]:
def transform(itr,tmx):
    '''
    Transform 3D-tensor (Euclidean or Cartesion tensor) of any order (>0) to another coordinate system
    Parameters:
        itr: input tensor, before transformation; should be a 3-element vector, a 3x3 matrix, or a 3x3x3x... multidimensional array, each dimension containing 3 elements
        tmx: transformation matrix, 3x3 matrix that contains the direction cosines between the old and the new coordinate system
    Returns:
        otr: output tensor, after transformation; has the same dimensions as the input tensor
    '''
    ne = np.asarray(itr).size
    nd = np.asarray(itr).ndim
    
    if (ne == 3):
        nd = 1
    
    otr = np.zeros_like(itr)
    
    iie = np.zeros((nd,1))
    ioe = np.zeros((nd,1))
    cne = np.cumprod(3 * np.ones((nd,1))) / 3
    
    for oe in range(ne):
        ioe = np.mod(np.floor(oe / cne).astype(int), 3)
        for ie in range(ne):
            pmx = 1
            iie = np.mod(np.floor(ie / cne).astype(int), 3)
            for i in range(nd):
                pmx = pmx * tmx[ioe[i], iie[i]]
            otr[np.unravel_index(oe, otr.shape)] = otr[np.unravel_index(oe, otr.shape)] + pmx * itr[np.unravel_index(ie, itr.shape)]
    
    return otr

In [66]:
def visual2D(c): 
    '''

    Parameters
    ----------
    c : 1D numpy array
        [C11, C12, C22, C16, C26, C66].

    Returns
    -------
    None.

    '''
    # transform c to CH (a 6x6 matrix)
    CH = np.ones((6,6)) * 1e-7
    CH[0,0] = c[0]
    CH[0,1] = c[1]
    CH[1,0] = c[1]
    CH[1,1] = c[2]
    CH[0,5] = c[3]
    CH[5,0] = c[3]
    CH[1,5] = c[4]
    CH[5,1] = c[4]
    CH[5,5] = c[5]
    # transform CH to a 3*3*3*3 tensor
    tensor = generate(CH)
    # find the E1 in 360 degree
    a = np.arange(0, 2*np.pi+0.02*np.pi, 0.02*np.pi)
    E1 = np.zeros_like(a)
    P1 = np.zeros_like(a)
    print(len(a))
    for i in range(len(a)):
        trans_z = np.array([[np.cos(a[i]), -np.sin(a[i]), 0],
                            [np.sin(a[i]),  np.cos(a[i]), 0],
                            [0,             0,            1]])
        # calculate the new tensor
        N_tensor = transform(tensor, trans_z)
        # transform the tensor to 6*6
        N_CH = ToMatrix(N_tensor)
        # calculate the E1
        E, P = modulus(N_CH)
        E1[i] = E[0]
        P1[i] = P[0,1]
    # x,y = pol2cart(a,E1)
#     print(E1, P1, E1.shape, P1.shape)
    return E1, P1
#     # Plot figure
#     fig, axs = plt.subplots(1, 2, subplot_kw={'projection': 'polar'})
#     axs[0].plot(a, E1)
#     axs[0].grid(True)
#     axs[0].set_title("Young's Modulus")
#     axs[1].plot(a, P1)
#     axs[1].grid(True)
#     axs[1].set_title("Poisson's Ratio")
#     plt.tight_layout()
#     plt.show()
    
    
def modulus(CH = None): 
    S = np.linalg.inv(CH[np.ix_([0,1,3], [0,1,3])])
    E = np.zeros((3,1))
    P = np.zeros((3,3))
    E[0] = 1 / S[0,0]
    E[1] = 1 / S[1,1]
    E[2] = 1 / S[2,2]
    P[0,1] = -S[0,1] * E[1];
    P[1,0] = -S[1,0] * E[0];
    P[2,1] = -S[1,2] * E[1];
    P[1,2] = -S[2,1] * E[2];
    return E, P
    
    
def generate(CH = None): 
    C = np.zeros((3,3,3,3))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            C[a,b,c,d] = CH[i,j]
    for i in range(3):
        if (i == 2):
            j = 0
        else:
            j = i + 1
        for m in range(3):
            if (m == 2):
                n = 0
            else:
                n = m + 1
            C[j,i,n,m] = C[i,j,m,n]
            C[j,i,m,n] = C[i,j,m,n]
            C[i,j,n,m] = C[i,j,m,n]
            C[j,i,m,m] = C[i,j,m,m]
            C[m,m,j,i] = C[m,m,i,j]
    return C
    
    
def change(w = None): 
    # change the index 4 5 6 to 23 31 12
    if (w < 3):
        a = w
        b = w
    else:
        if (w == 3):
            a = 1
            b = 2
        else:
            if (w == 4):
                a = 2
                b = 0
            else:
                if (w == 5):
                    a = 0
                    b = 1
    return a,b
    
    
def ToMatrix(C = None): 
    CH = np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            a,b = change(i)
            c,d = change(j)
            CH[i,j] = C[a,b,c,d]
    return CH

In [67]:
import time 

start = time.time() 
c = np.array([604591262.41, 102330968.98, 1157840534.65, 0.00, 0.00, 156833419.95])
visual2D(c)
print(time.time() - start)

101
8.684705018997192


In [68]:
small = sub.head(n=3)

In [69]:
#[C11, C12, C22, C16, C26, C66]
start = time.time()
res = small.apply(lambda row : visual2D([row['C11'], row["C12"], row['C22'], row['C16'], row['C26'], row['C66']]), axis=1)
print("Time for computing %d rows: %f"%(sub.shape[0], time.time() - start))


101
101
101
Time for computing 158 rows: 25.684371


In [70]:
sub['youngs'] = res.apply(lambda x : x[0])
sub['poisson'] = res.apply(lambda x : x[1])

In [71]:
sub.head()

,symmetry,unit_cell_x_pixels,unit_cell_y_pixels,geometry_condensed,geometry_full,condition,C11,C12,C22,C16,...,CM0_C66,CM1,CM1_C11,CM1_C12,CM1_C22,CM1_C16,CM1_C26,CM1_C66,youngs,poisson
54099,C2v,50,50,NaN,0000000000000000000111111111111000000000000000...,Plane Strain,6.180045e+08,111902107.9,7.405735e+08,0,...,0,vero_white,2963290579,1459531181,2963290579,0,0,751879699,"[601095866.8599133, 596686054.5589361, 5834598...","[0.18107005482437127, 0.18796926662730962, 0.2..."
34368,C2v,50,50,NaN,0000000000001111111111111111111111111100000000...,Plane Strain,6.693100e+08,202799631.2,1.241218e+09,0,...,0,vero_white,2963290579,1459531181,2963290579,0,0,751879699,"[636175026.970261, 634783694.2489499, 63068740...","[0.3029980692272017, 0.3071238199050367, 0.319..."
1916,C2v,50,50,NaN,1111111111111111111111111111111111111111111111...,Plane Strain,1.755271e+09,588758063.8,1.928755e+09,0,...,0,vero_white,2963290579,1459531181,2963290579,0,0,751879699,"[1575550905.0605085, 1575498386.3415282, 15753...","[0.33542287526379944, 0.33559245605619203, 0.3..."
46212,C2v,50,50,NaN,1111111111111111111111111111111111111111111111...,Plane Strain,5.173402e+08,156942733.7,1.236537e+09,0,...,0,vero_white,2963290579,1459531181,2963290579,0,0,751879699,NaN,NaN
65912,C2v,50,50,NaN,1111111111111111111111111111111111111111111111...,Plane Strain,7.444451e+08,160455366.8,9.518284e+08,0,...,0,vero_white,2963290579,1459531181,2963290579,0,0,751879699,NaN,NaN


In [72]:
sub.to_csv('data/smallsample2.csv')